In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [4]:
# Loading the dataset
df = pd.read_csv("C:/Users/User/Downloads/voice.csv")

In [5]:
# Extracting features and labels
X = df.iloc[:, :-1]  # Assuming the last column is the label
y = df['label']

# Converting the gender labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
# Useing only the first 13 features since audio file contains 13 features
X = X.iloc[:, :13]

# Spliting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Training a machine learning model using random forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# Makeing predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

# Saving the trained model and label encoder for use later in the program
joblib.dump(model, 'gender_detection_model.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

Test Accuracy: 0.9794952681388013


['label_encoder.pkl']

In [8]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import librosa
import joblib
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
# Load the pre-trained gender detection model from random forest 
model = joblib.load("gender_detection_model.pkl")

In [11]:
def process_audio(file_path):
    try:
        # Load and preprocess audio file
        audio, _ = librosa.load(file_path, sr=16000, duration=3)
        mfccs = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=13)
        mfccs_mean = np.mean(mfccs, axis=1)

        # Make a prediction
        prediction = model.predict([mfccs_mean])
        gender = "Female" if prediction[0] > 0.5 else "Male"

        # Display the result
        result_label.config(text=f"Detected Gender: {gender}")
        file_name_label.config(text=f"File Name: {file_path}")

    except Exception as e:
        messagebox.showerror("Error", f"Error processing audio: {str(e)}")
        
def play_audio(file_path):
    try:
        pygame.mixer.init()
        pygame.mixer.music.load(file_path)
        pygame.mixer.music.play()
    except Exception as e:
        messagebox.showerror("Error", f"Error playing audio: {str(e)}")

def reset_result():
    result_label.config(text="Detected Gender: ")
    file_name_label.config(text="File Name: ")

def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Audio files", "*.wav;*.mp3")])
    if file_path:
        process_audio(file_path)

In [12]:
# GUI setup
app = tk.Tk()
app.title("Gender Detection App")

''

In [13]:
# Create widgets
browse_button = tk.Button(app, text="Browse", command=browse_file)
play_button = tk.Button(app, text="Play Audio", command=lambda: play_audio(filedialog.askopenfilename()))
reset_button = tk.Button(app, text="Reset", command=reset_result)
result_label = tk.Label(app, text="Detected Gender: ")
file_name_label = tk.Label(app, text="File Name: ")

In [14]:
# Place widgets in the GUI
browse_button.pack(pady=10)
play_button.pack(pady=10)
reset_button.pack(pady=10)
result_label.pack(pady=10)
file_name_label.pack(pady=10)

In [ ]:
# Run the GUI
app.mainloop()